# Exporting ResNet18 from Pytorch to ONNX format. 

In [ ]:
import torch 
from torchvision.models import resnet18, ResNet18_Weights

## Load the pre-trained model

In [ ]:
# weights = ResNet18_Weights.DEFAULT
# resnet18 = resnet18(weights=weights)

## alternative

In [ ]:
import torch 
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)


# Provide random input tensor

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)

ResNet18 classification model batched (B, C, H, W) 

## Export the model to onnx format
<br>
Assign first the input and output names

In [ ]:
torch.onnx.export(resnet18,
                  dummy_input,
                  "resnet18.onnx",
                  input_names=["input"],
                  output_names=["output"])

## Verify ONNX Model

In [ ]:
import onnxruntime as ort

resnet18_onnx = ort.InferenceSession("resnet18.onnx")

print("Input names:", resnet18_onnx.get_inputs()[0].name)
print("Output names:", resnet18_onnx.get_outputs()[0].name)

# Verification of the Exported Model Using Netron

<img src="input_resnet18.png" alt="Input of ResNet18 in Netron" width="1000" height="600">

<img src="output_resnet18.png" alt="Output of ResNet18 in Netron" width="1000" height="600">

## Using ONNXRUNTIME

Implementing ResNet18 on the CPU involves severar implementation steps

- Preparation of the input tensor (preproceccing)
- Performing an inference in the CPU accelerator
- Converting class probabilities to class labels (postprocessing)
- Visualization 

<img src="parrot.jpg" width="400" height="400">

In [ ]:
import numpy as np
import cv2




def preprocessing(image): 
    
    img = cv2.resize(image, (256, 256))
    height, width, _ = img.shape
    x = (width - 224)//2
    y = (height - 224)//2
    img = img[y:y+224, x:x+224]
    img = np.asarray(img, np.float32)
    img = np.transpose(img, [2, 0, 1])
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    for i in range(3):
        img[i,:,:] = (img[i,:,:]/255 - mean[i] / std[i])
    
    img = np.expand_dims(img, axis=0)
    
    return img


img = "parrot.jpg"
img = cv2.imread(img)
img = preprocessing(img)


In [ ]:
import onnxruntime as ort

session_options = ort.SessionOptions()
session = ort.InferenceSession("resnet18.onnx", session_options, ["CPUExecutionProvider"])


input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
ortvalue = ort.OrtValue.ortvalue_from_numpy(img)
session = session.run([output_name], {input_name:ortvalue})

In [19]:
import os 


def read_classes(filepath):
    with open(filepath, 'r') as file: 
        classes = [line.strip() for line in file]
    return classes
        



def postprocessing(data, classes):
    
    data = np.exp(data - np.max(data))
    prob = data/data.sum()
    pred_index = np.argmax(prob)
    predicted_class = classes[pred_index]
    
    return predicted_class, pred_index
    

classes = read_classes(os.path.abspath('../classes.txt'))

predicted_class, predicted_index = postprocessing(session, classes)

print("Predicted Class:", predicted_class)
print("Predicted Index:", predicted_index)
    

Predicted Class: sulphur-crested cockatoo
Predicted Index: 89


Implementation of ResNet18 on the GPU. In case you installed `onnxruntime`, you need uninstall onnxruntime and install `pip install onnxruntime-gpu`. 

In [ ]:
import onnxruntime as ort
import torch

providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']



session_options = ort.SessionOptions()
session_gpu = ort.InferenceSession("resnet18.onnx", session_options, providers)
input_name = session_gpu.get_inputs()[0].name
output_name = session_gpu.get_outputs()[0].name
io_binding = session_gpu.io_binding()
ortvalue = ort.OrtValue.ortvalue_from_numpy(img, 'cuda', 0)
io_binding.bind_input(name=input_name,
                        device_type='cuda',
                        device_id=0,
                        element_type=np.float32,
                        shape = ortvalue.shape(),
                        buffer_ptr=ortvalue.data_ptr()
                        )

io_binding.bind_output(name= output_name,
                             device_type='cuda',
                             device_id=0)

session_gpu.run_with_iobinding(io_binding)

output = io_binding.copy_outputs_to_cpu()
predicted_class, predicted_index = postprocessing(output, classes)

print("Predicted Class:", predicted_class)
print("Predicted Index:", predicted_index)